In [1]:
# !pip install kaggle
# !mkdir -p ~/.kaggle/
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [2]:
# !kaggle datasets download -d alincijov/pix2pix-maps
# !unzip pix2pix-maps.zip

In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

val_filenames = os.listdir('val')
val_filenames = ['val/' + filename for filename in val_filenames]
train_filenames = os.listdir('train')
train_filenames = ['train/' + filename for filename in train_filenames]

filenames = val_filenames + train_filenames

In [7]:
len(filenames)

2195

In [8]:
import os
from PIL import Image
import numpy as np
from tqdm import tqdm

def split_into_4_crops(image_array):
    """
    Split the given image array into 4 crops: top left, top right, bottom left, bottom right.
    """
    height, width, _ = image_array.shape
    half_height = height // 2
    half_width = width // 2

    top_left = image_array[:half_height, :half_width, :]
    top_right = image_array[:half_height, half_width:, :]
    bottom_left = image_array[half_height:, :half_width, :]
    bottom_right = image_array[half_height:, half_width:, :]

    return top_left, top_right, bottom_left, bottom_right

sat_paths = []
map_paths = []
indexes = []

os.makedirs('all_data_npy', exist_ok=True)
os.makedirs('all_data_npy/sat/', exist_ok=True)
os.makedirs('all_data_npy/map/', exist_ok=True)

for i, filename in tqdm(enumerate(filenames), total=len(filenames)):
    im = Image.open(filename)
    im_array = np.array(im)

    sat = im_array[:, :600, :]
    map_img = im_array[:, 600:, :]


    sat_crops = split_into_4_crops(sat)
    map_crops = split_into_4_crops(map_img)

    for j, (sat_crop, map_crop) in enumerate(zip(sat_crops, map_crops)):
        unique_index = i * 4 + j

        sat_path = 'all_data_npy/sat/' + str(unique_index) + '.npy'
        map_path = 'all_data_npy/map/' + str(unique_index) + '.npy'

        np.save(sat_path, sat_crop)
        np.save(map_path, map_crop)

        indexes.append(unique_index)
        sat_paths.append(sat_path)
        map_paths.append(map_path)

100%|█████████▉| 2194/2195 [02:28<00:00, 14.80it/s]


IsADirectoryError: [Errno 21] Is a directory: 'train/.ipynb_checkpoints'

In [9]:
metadata = pd.DataFrame()

In [10]:
metadata['id'] = indexes
metadata['sat_path'] = sat_paths
metadata['map_path'] = map_paths

In [14]:
metadata['sat_path'] = 'data/SATELLITE/' + metadata['sat_path']
metadata['map_path'] = 'data/SATELLITE/' + metadata['map_path']

In [17]:
metadata.head()

,id,sat_path,map_path
0,0,data/SATELLITE/all_data_npy/sat/0.npy,data/SATELLITE/all_data_npy/map/0.npy
1,1,data/SATELLITE/all_data_npy/sat/1.npy,data/SATELLITE/all_data_npy/map/1.npy
2,2,data/SATELLITE/all_data_npy/sat/2.npy,data/SATELLITE/all_data_npy/map/2.npy
3,3,data/SATELLITE/all_data_npy/sat/3.npy,data/SATELLITE/all_data_npy/map/3.npy
4,4,data/SATELLITE/all_data_npy/sat/4.npy,data/SATELLITE/all_data_npy/map/4.npy


In [16]:
metadata.to_csv('metadata.csv', index=False)